In [ ]:
#!pip install bokeh
!pip install dash

In [1]:
import dash, os
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [2]:


# Carregar os dados
path = 'D:/Disco/Data/PUCRS/RANDON'
estoque = pd.read_csv(f'{path}/f_estoque.csv')
entrada = pd.read_csv(f'{path}/f_entrada.csv')

files = [f for f in os.listdir(f'{path}/f_demanda') if f.endswith('.csv')]
demanda = pd.concat([pd.read_csv(os.path.join(f'{path}/f_demanda', f)) for f in files])

material = pd.read_csv(f'{path}/d_material.csv')
fornecedor = pd.read_csv(f'{path}/d_fornecedor.csv')

# Preparação dos dados
estoque['cod_material'] = estoque['cod_material'].astype(str)
entrada['cod_material'] = entrada['cod_material'].astype(str)
entrada['dt_data_remessa'] = pd.to_datetime(entrada['dt_data_remessa'])
demanda['dt_necessidade'] = pd.to_datetime(demanda['dt_necessidade'])
material['cod_material'] = material['cod_material'].astype(str)
fornecedor['cod_fornecedor'] = fornecedor['cod_fornecedor'].astype(str)

# Merge para análise
entrada_fornecedor = entrada.merge(fornecedor, on='cod_fornecedor')
demanda_material = demanda.merge(material, on='cod_material')
entrada_estoque = entrada.merge(estoque, on='cod_material')
fornecedor_material = entrada.merge(fornecedor, on='cod_fornecedor').merge(material, on='cod_material')

In [ ]:
entrada_fornecedor

In [ ]:
fornecedor_material

In [ ]:
estoque

In [ ]:
demanda

In [3]:


# Leitura dos dados
path = 'D:/Disco/Data/PUCRS/RANDON'
fornecedores = pd.read_csv(f'{path}/d_fornecedor.csv')
transacoes = pd.read_csv(f'{path}/f_entrada.csv')
estoque = pd.read_csv(f'{path}/f_estoque.csv')

# Conversão de datas
transacoes['data'] = pd.to_datetime(transacoes['dt_data_remessa'])

# Inicializando o aplicativo Dash
app = dash.Dash(__name__)

# Layout do Dashboard
app.layout = html.Div([
    html.H1("Dashboard de Fornecedores"),
    
    # Visão Geral dos Fornecedores
    html.Div([
        html.H2("Visão Geral dos Fornecedores"),
        dcc.Graph(
            id='total-fornecedores',
            figure={
                'data': [{'x': ['Total'], 'y': [len(fornecedores)], 'type': 'bar'}],
                'layout': {'title': 'Número Total de Fornecedores'}
            }
        ),
        dcc.Graph(
            id='distribuicao-categoria',
            figure=px.pie(fornecedores, names='nm_regiao', title='Distribuição por Região')
        ),
        dcc.Graph(
            id='materiais-fornecidos',
            figure=px.bar(transacoes, x='fornecedor_id', y='quantidade', title='Quantidade de Materiais Fornecidos')
        )
    ]),

    # Análise de Transações
    html.Div([
        html.H2("Análise de Transações"),
        dcc.Graph(
            id='transacoes-tempo',
            figure=px.line(transacoes, x='data', y='quantidade', title='Transações ao Longo do Tempo')
        ),
        dcc.Graph(
            id='transacoes-categoria',
            figure=px.bar(transacoes, x='data', y='quantidade', title='Transações por Categoria')
        )
    ]),

    # Desempenho por Fornecedor
    html.Div([
        html.H2("Desempenho por Fornecedor"),
        dcc.Graph(
            id='top-fornecedores',
            figure=px.bar(transacoes.groupby('fornecedor_id')['quantidade'].sum().reset_index(), x='fornecedor_id', y='quantidade', title='Top Fornecedores')
        ),
        dcc.Graph(
            id='frequencia-transacoes',
            figure=px.scatter(transacoes, x='fornecedor_id', y='quantidade', title='Frequência de Transações')
        )
    ]),

    # Mapeamento Geográfico dos Fornecedores
    html.Div([
        html.H2("Mapeamento Geográfico dos Fornecedores"),
        dcc.Graph(
            id='mapa-fornecedores',
            figure=px.scatter_mapbox(fornecedores, lat='nr_media_delivery_days', lon='nr_delivery_days_FOB', hover_name='nm_local', mapbox_style="carto-positron", title='Mapa de Fornecedores')
        )
    ])
])


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['cod_documento_compras', 'cod_item_documento_compras', 'cod_divisao_remessa', 'cod_material', 'cod_incoterm_01', 'cod_fornecedor', 'dt_data_remessa', 'nr_qtd_necessidade', 'data'] but received: fornecedor_id